<a href="https://colab.research.google.com/github/endteamschoolofai/END/blob/main/Session9/Assignment_9_Learning_Phrase_Representations_using_RNN_Encoder_Decoder_for_Statistical_Machine_Translation_dataset_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation

In this second notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078). This model will achieve improved test perplexity whilst only using a single layer RNN in both the encoder and the decoder.

## Introduction

Let's remind ourselves of the general encoder-decoder model.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

We use our encoder (green) over the embedded source sequence (yellow) to create a context vector (red). We then use that context vector with the decoder (blue) and a linear layer (purple) to generate the target sentence.

In the previous model, we used an multi-layered LSTM as the encoder and decoder.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq4.png?raw=1)

One downside of the previous model is that the decoder is trying to cram lots of information into the hidden states. Whilst decoding, the hidden state will need to contain information about the whole of the source sequence, as well as all of the tokens have been decoded so far. By alleviating some of this information compression, we can create a better model!

We'll also be using a GRU (Gated Recurrent Unit) instead of an LSTM (Long Short-Term Memory). Why? Mainly because that's what they did in the paper (this paper also introduced GRUs) and also because we used LSTMs last time. To understand how GRUs (and LSTMs) differ from standard RNNS, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) link. Is a GRU better than an LSTM? [Research](https://arxiv.org/abs/1412.3555) has shown they're pretty much the same, and both are better than standard RNNs. 

## Preparing Data

All of the data preparation will be (almost) the same as last time, so we'll very briefly detail what each code block does. See the previous notebook for a recap.

We'll import PyTorch, TorchText, spaCy and a few standard modules.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext import data 
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
import pandas as pd

Then set a random seed for deterministic results/reproducability.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset 2 - CommonsenseQA 

In [4]:
!wget https://s3.amazonaws.com/commensenseqa/train_rand_split.jsonl
!wget https://s3.amazonaws.com/commensenseqa/dev_rand_split.jsonl
!wget https://s3.amazonaws.com/commensenseqa/test_rand_split_no_answers.jsonl

--2021-01-08 16:12:37--  https://s3.amazonaws.com/commensenseqa/train_rand_split.jsonl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.43.158
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.43.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785890 (3.6M) [binary/octet-stream]
Saving to: ‘train_rand_split.jsonl’

train_rand_split.js 100%[===================>]   3.61M  1.83MB/s    in 2.0s    

2021-01-08 16:12:40 (1.83 MB/s) - ‘train_rand_split.jsonl’ saved [3785890/3785890]

--2021-01-08 16:12:40--  https://s3.amazonaws.com/commensenseqa/dev_rand_split.jsonl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.43.158
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.43.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 471653 (461K) [binary/octet-stream]
Saving to: ‘dev_rand_split.jsonl’

dev_rand_split.json 100%[===================>] 460.60K   641KB/s    in 0.7s    

2021-01-08 16:12:42 (641 KB/s) - ‘d

In [5]:
train_df_second = pd.read_json('train_rand_split.jsonl', lines = True)
valid_df_second = pd.read_json('dev_rand_split.jsonl', lines = True)
test_df_second = pd.read_json('test_rand_split_no_answers.jsonl', lines = True)

In [6]:
train_df_second.head()

,answerKey,id,question
0,A,075e483d21c29a511267ef62bedc0461,"{'question_concept': 'punishing', 'choices': [..."
1,B,61fe6e879ff18686d7552425a36344c8,"{'question_concept': 'people', 'choices': [{'l..."
2,A,4c1cb0e95b99f72d55c068ba0255c54d,"{'question_concept': 'choker', 'choices': [{'l..."
3,D,02e821a3e53cb320790950aab4489e85,"{'question_concept': 'highway', 'choices': [{'..."
4,C,23505889b94e880c3e89cff4ba119860,"{'question_concept': 'fox', 'choices': [{'labe..."


In [7]:
valid_df_second.head()

,answerKey,id,question
0,A,1afa02df02c908a558b4036e80242fac,"{'question_concept': 'revolving door', 'choice..."
1,A,a7ab086045575bb497933726e4e6ad28,"{'question_concept': 'people', 'choices': [{'l..."
2,B,b8c0a4703079cf661d7261a60a1bcbff,"{'question_concept': 'magazines', 'choices': [..."
3,A,e68fb2448fd74e402aae9982aa76e527,"{'question_concept': 'hamburger', 'choices': [..."
4,A,2435de612dd69f2012b9e40d6af4ce38,"{'question_concept': 'farmland', 'choices': [{..."


In [8]:
test_df_second.head()

,id,question
0,90b30172e645ff91f7171a048582eb8b,"{'question_concept': 'townhouse', 'choices': [..."
1,000990552527b1353f98f1e1a7dfc643,"{'question_concept': 'star', 'choices': [{'lab..."
2,dca0f2859f3c3dd43a9b2bfeff4936a8,"{'question_concept': 'kids', 'choices': [{'lab..."
3,8795a949b39702af0e452c9e1229046d,"{'question_concept': 'person', 'choices': [{'l..."
4,1f74ea1f73b9f5d91a665b4d90218a6e,"{'question_concept': 'ignorance', 'choices': [..."


In [9]:
len(train_df_second), len(valid_df_second), len(test_df_second)

(9741, 1221, 1140)

In [10]:
train_df_second.question[0], train_df_second.answerKey[0]

({'choices': [{'label': 'A', 'text': 'ignore'},
   {'label': 'B', 'text': 'enforce'},
   {'label': 'C', 'text': 'authoritarian'},
   {'label': 'D', 'text': 'yell at'},
   {'label': 'E', 'text': 'avoid'}],
  'question_concept': 'punishing',
  'stem': 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?'},
 'A')

In [11]:
valid_df_second.question[0], valid_df_second.answerKey[0]

({'choices': [{'label': 'A', 'text': 'bank'},
   {'label': 'B', 'text': 'library'},
   {'label': 'C', 'text': 'department store'},
   {'label': 'D', 'text': 'mall'},
   {'label': 'E', 'text': 'new york'}],
  'question_concept': 'revolving door',
  'stem': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?'},
 'A')

In [12]:
test_df_second.question[0]

{'choices': [{'label': 'A', 'text': 'suburban development'},
  {'label': 'B', 'text': 'apartment building'},
  {'label': 'C', 'text': 'bus stop'},
  {'label': 'D', 'text': 'michigan'},
  {'label': 'E', 'text': 'suburbs'}],
 'question_concept': 'townhouse',
 'stem': 'The townhouse was a hard sell for the realtor, it was right next to a high rise what?'}

In [13]:
choice_labels = [train_df_second.question[0]['choices'][i]['label'] for i in range(len(train_df_second.question[0]['choices']))] #for j in range(1)]
choice_labels

['A', 'B', 'C', 'D', 'E']

In [14]:
train_label_pos = [choice_labels.index(train_df_second.answerKey[i]) for i in range(len(train_df_second))]
valid_label_pos = [choice_labels.index(valid_df_second.answerKey[j]) for j in range(len(valid_df_second))]

In [15]:
train_questions_second = [train_df_second.question[i]['stem'] for i in range(len(train_df_second))]
train_answers_second = [train_df_second.question[i]['choices'][train_label_pos[i]]['text'] for i in range(len(train_df_second))]

valid_questions_second = [valid_df_second.question[i]['stem'] for i in range(len(valid_df_second))]
valid_answers_second = [valid_df_second.question[i]['choices'][valid_label_pos[i]]['text'] for i in range(len(valid_df_second))]

test_questions_second = [test_df_second.question[i]['stem'] for i in range(len(test_df_second))]

train_questions_second[:5], train_answers_second[:5], valid_questions_second[:5], valid_answers_second[:5], test_questions_second[:5] 

(['The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?',
  'Sammy wanted to go to where the people were.  Where might he go?',
  'To locate a choker not located in a jewelry box or boutique where would you go?',
  'Google Maps and other highway and street GPS services have replaced what?',
  'The fox walked from the city into the forest, what was it looking for?'],
 ['ignore', 'populated areas', 'jewelry store', 'atlas', 'natural habitat'],
 ['A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?',
  'What do people aim to do at work?',
  'Where would you find magazines along side many other printed works?',
  'Where are  you likely to find a hamburger?',
  'James was looking for a good place to buy farmland.  Where might he look?'],
 ['bank', 'complete job', 'bookstore', 'fast food restaurant', 'midwest'],
 ['The townhouse was a hard sell for the realtor, it wa

In [16]:
Questions_Second = data.Field(sequential = True, tokenize = 'spacy', batch_first =True)#, include_lengths=True)
Answers_Second = data.Field(tokenize ='spacy', is_target=True, batch_first =True)#, sequential =False)

In [17]:
fields = [('questions_second', Questions_Second),('answers_second', Answers_Second)]

In [18]:
example1 = [data.Example.fromlist([train_questions_second[i], train_answers_second[i]], fields) for i in range(len(train_df_second))] 
example2 = [data.Example.fromlist([valid_questions_second[i], valid_answers_second[i]], fields) for i in range(len(valid_df_second))] 
example3 = [data.Example.fromlist([test_questions_second[i], None], fields) for i in range(len(test_df_second))] 

In [19]:
Train_Dataset_Second = data.Dataset(example1, fields)
Valid_Dataset_Second = data.Dataset(example2, fields)
Test_Dataset_Second = data.Dataset(example3, fields)

In [20]:
len(Train_Dataset_Second), len(Valid_Dataset_Second), len(Test_Dataset_Second)

(9741, 1221, 1140)

In [21]:
print(vars(Train_Dataset_Second.examples[1]))
print(vars(Valid_Dataset_Second.examples[1]))
print(vars(Test_Dataset_Second.examples[1]))

{'questions_second': ['Sammy', 'wanted', 'to', 'go', 'to', 'where', 'the', 'people', 'were', '.', ' ', 'Where', 'might', 'he', 'go', '?'], 'answers_second': ['populated', 'areas']}
{'questions_second': ['What', 'do', 'people', 'aim', 'to', 'do', 'at', 'work', '?'], 'answers_second': ['complete', 'job']}
{'questions_second': ['There', 'is', 'a', 'star', 'at', 'the', 'center', 'of', 'what', 'group', 'of', 'celestial', 'bodies', '?'], 'answers_second': None}


In [22]:
Questions_Second.build_vocab(Train_Dataset_Second)
Answers_Second.build_vocab(Train_Dataset_Second)

In [23]:
print('Size of input vocab : ', len(Questions_Second.vocab))
print('Size of output vocab : ', len(Answers_Second.vocab))
print('Top 10 words appreared repeatedly :', list(Questions_Second.vocab.freqs.most_common(10)))
print('Labels : ', Answers_Second.vocab.stoi)

Size of input vocab :  8634
Size of output vocab :  3821
Top 10 words appreared repeatedly : [('?', 9680), ('a', 5310), ('to', 5086), ('the', 3972), ('what', 3871), (',', 3641), ('you', 2613), ('What', 2330), ('is', 2251), ('was', 2208)]
Labels :  defaultdict(<function _default_unk_index at 0x7f3d74d83f28>, {'<unk>': 0, '<pad>': 1, 'store': 2, 'house': 3, 'to': 4, 'of': 5, "'s": 6, 'office': 7, 'room': 8, 'city': 9, 'get': 10, 'building': 11, 'school': 12, 'money': 13, 'new': 14, 'have': 15, 'go': 16, 'home': 17, 'in': 18, 'music': 19, 'being': 20, 'water': 21, 'down': 22, 'food': 23, 'feel': 24, 'park': 25, 'fun': 26, 'area': 27, 'own': 28, 'restaurant': 29, 'shop': 30, 'work': 31, 'good': 32, 'game': 33, 'kitchen': 34, 'cabinet': 35, 'countryside': 36, 'out': 37, 'people': 38, 'make': 39, 'town': 40, 'getting': 41, 'table': 42, 'apartment': 43, 'desk': 44, 'play': 45, 'better': 46, 'tired': 47, 'up': 48, 'for': 49, 'michigan': 50, 'death': 51, 'feeling': 52, 'hotel': 53, 'market': 54

In [24]:
BATCH_SIZE = 128
train_iterator_second, valid_iterator_second, test_iterator_second = BucketIterator.splits(
                                (Train_Dataset_Second, Valid_Dataset_Second, Test_Dataset_Second),  
                                batch_size = BATCH_SIZE,
                                sort_key = lambda x: len(x.questions_second),
                                sort_within_batch=True ,
                                device = device)



In [25]:
for i, batch in enumerate(valid_iterator_second):
  print(type(batch), type(batch.answers_second), type(batch.questions_second))
  break

<class 'torchtext.data.batch.Batch'> <class 'torch.Tensor'> <class 'torch.Tensor'>


## Building the Seq2Seq Model

### Encoder

The encoder is similar to the previous one, with the multi-layer LSTM swapped for a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). Again, for more info, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent post. 

The rest of the encoder should be very familar from the last session, it takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, passes it through the embedding layer, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

This is identical to the encoder of the general seq2seq model, with all the "magic" happening inside the GRU (green).

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq5.png?raw=1)

In [26]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

## Decoder

The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression.

Instead of the GRU in the decoder taking just the embedded target token, $d(y_t)$ and the previous hidden state $s_{t-1}$ as inputs, it also takes the context vector $z$. 

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Note how this context vector, $z$, does not have a $t$ subscript, meaning we re-use the same context vector returned by the encoder for every time-step in the decoder. 

Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Thus, our decoder now looks something like this:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq6.png?raw=1)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.

How do these two changes reduce the information compression? Well, hypothetically the decoder hidden states, $s_t$, no longer need to contain information about the source sequence as it is always available as an input. Thus, it only needs to contain information about what tokens it has generated so far. The addition of $y_t$ to the linear layer also means this layer can directly see what the token is, without having to get this information from the hidden state. 

However, this hypothesis is just a hypothesis, it is impossible to determine how the model actually uses the information provided to it (don't listen to anyone that says differently). Nevertheless, it is a solid intuition and the results seem to indicate that this modifications are a good idea!

Within the implementation, we will pass $d(y_t)$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $d(y_t), s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. We also don't pass a value of dropout to the GRU as it only uses a single layer.

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $d(y_t)$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [27]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [28]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

# Training the Seq2Seq Model

The rest of this session is very similar to the previous one. 

We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [29]:
INPUT_DIM = len(Questions_Second.vocab)
OUTPUT_DIM = len(Answers_Second.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [30]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(8634, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(3821, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=3821, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

We print out the number of parameters.

Even though we only have a single layer RNN for our encoder and decoder we actually have **more** parameters  than the last model. This is due to the increased size of the inputs to the GRU and the linear layer. However, it is not a significant amount of parameters and causes a minimal amount of increase in training time (~3 seconds per epoch extra).

In [31]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,235,053 trainable parameters


We initiaize our optimizer.

In [32]:
optimizer = optim.Adam(model.parameters())

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [33]:
TRG_PAD_IDX = Answers_Second.vocab.stoi[Answers_Second.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [39]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.questions_second
        trg = batch.answers_second
        
        src = src.permute(1, 0)
        trg = trg.permute(1, 0)

        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:]
        trg = torch.reshape(trg, (-1,)) #trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [38]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.questions_second
            trg = batch.answers_second
            
            src = src.permute(1, 0)
            trg = trg.permute(1, 0)

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:]
            trg = torch.reshape(trg, (-1,)) #trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [36]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [40]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator_second, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator_second, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 54s
	Train Loss: 7.035 | Train PPL: 1135.915
	 Val. Loss: 6.565 |  Val. PPL: 709.561
Epoch: 02 | Time: 0m 55s
	Train Loss: 6.029 | Train PPL: 415.394
	 Val. Loss: 6.385 |  Val. PPL: 593.117
Epoch: 03 | Time: 0m 54s
	Train Loss: 5.485 | Train PPL: 240.986
	 Val. Loss: 5.979 |  Val. PPL: 395.019
Epoch: 04 | Time: 0m 54s
	Train Loss: 4.866 | Train PPL: 129.810
	 Val. Loss: 5.928 |  Val. PPL: 375.242
Epoch: 05 | Time: 0m 58s
	Train Loss: 4.276 | Train PPL:  71.935
	 Val. Loss: 5.979 |  Val. PPL: 395.212
Epoch: 06 | Time: 0m 58s
	Train Loss: 3.728 | Train PPL:  41.615
	 Val. Loss: 5.724 |  Val. PPL: 306.121
Epoch: 07 | Time: 0m 55s
	Train Loss: 3.166 | Train PPL:  23.702
	 Val. Loss: 5.856 |  Val. PPL: 349.310
Epoch: 08 | Time: 0m 54s
	Train Loss: 2.657 | Train PPL:  14.255
	 Val. Loss: 6.009 |  Val. PPL: 407.119
Epoch: 09 | Time: 0m 53s
	Train Loss: 2.181 | Train PPL:   8.859
	 Val. Loss: 6.213 |  Val. PPL: 499.393
Epoch: 10 | Time: 0m 53s
	Train Loss: 1.779 | Train PP

Finally, we test the model on the test set using these "best" parameters.

In [ ]:
#model.load_state_dict(torch.load('tut2-model.pt'))

#test_loss = evaluate(model, test_iterator, criterion)

#print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Just looking at the test loss, we get better performance. This is a pretty good sign that this model architecture is doing something right! Relieving the information compression seems like the way forard, and in the next tutorial we'll expand on this even further with *attention*.